In [ ]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools

In [ ]:
class FullbandModel(nn.Module):
    def __init__(self, freq_bins, time_bins, hidden_dim, beta=0.9):
        """
        Fullband Model with normalization, spiking neuron layer, and a linear layer.
        Args:
        - freq_bins: Number of frequency bins in the spectrogram.
        - time_bins: Number of time bins in the spectrogram.
        - hidden_dim: Number of hidden units for the spiking neuron layer.
        - beta: Decay parameter for the LIF neuron.
        """
        super(FullbandModel, self).__init__()
        
        self.freq_bins = freq_bins
        self.time_bins = time_bins
        self.input_dim = freq_bins * time_bins  # Flattened input feature size
        self.hidden_dim = hidden_dim

        # Layers
        self.normalization = nn.LayerNorm(self.input_dim)  # Normalize input features
        self.spikingneuron = snn.Leaky(beta=beta)          # Leaky Integrate-and-Fire neuron
        self.linear = nn.Linear(self.input_dim, self.hidden_dim)  # Linear transformation

    def forward(self, x, num_steps=10):
        """
        Forward pass for the FullbandModel with time-stepped spiking neuron dynamics.
        Args:
        - x: Input tensor of shape (batch_size, frequency_bins, time_bins).
        - num_steps: Number of time steps for spiking neuron simulation.
        Returns:
        - spk_rec: Spiking activity across timesteps (num_steps, batch_size, hidden_dim).
        - mem_rec: Membrane potential across timesteps (num_steps, batch_size, hidden_dim).
        """
        batch_size, freq_bins, time_bins = x.shape
        assert freq_bins == self.freq_bins and time_bins == self.time_bins, \
            "Input dimensions must match model initialization dimensions."

        # Flatten the spectrogram into 1D vectors
        x = x.view(batch_size, -1)  # Shape: (batch_size, input_dim)

        # Normalize the input
        x = self.normalization(x)

        # Initialize membrane potentials for the spiking neuron
        mem = torch.zeros((batch_size, self.hidden_dim), dtype=torch.float32, device=x.device)

        # Record spiking activity and membrane potentials
        spk_rec = []
        mem_rec = []

        for step in range(num_steps):
            # Linear transformation
            cur = self.linear(x)

            # Spiking neuron dynamics
            spk, mem = self.spikingneuron(cur, mem)

            # Record outputs
            spk_rec.append(spk)
            mem_rec.append(mem)

        # Stack outputs across timesteps
        spk_rec = torch.stack(spk_rec, dim=0)  # Shape: (num_steps, batch_size, hidden_dim)
        mem_rec = torch.stack(mem_rec, dim=0)  # Shape: (num_steps, batch_size, hidden_dim)

        return spk_rec, mem_rec

In [ ]:
class SubbandModel(nn.Module):
    def __init__(self, hidden_dim, num_steps, beta=0.9):
        """
        Subband Model with normalization, spiking neuron layer, and a linear layer.
        Args:
        - hidden_dim: Number of hidden units for the spiking neuron layer.
        - num_steps: Number of timesteps for spiking neuron simulation.
        - beta: Decay parameter for the LIF neuron.
        """
        super(SubbandModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_steps = num_steps
        self.spikingneuron = snn.Leaky(beta=beta)  # Leaky Integrate-and-Fire neuron
        self.linear = None  # Linear layer to be initialized dynamically

    def forward(self, x, num_steps=None):
        """
        Forward pass for the SubbandModel with time-stepped spiking neuron dynamics.
        Args:
        - x: Input tensor of shape (batch_size, subband_dim).
        - num_steps: Number of time steps for spiking neuron simulation.
        Returns:
        - spk_rec: Spiking activity across timesteps (num_steps, batch_size, hidden_dim).
        - mem_rec: Membrane potential across timesteps (num_steps, batch_size, hidden_dim).
        """
        if num_steps is None:
            num_steps = self.num_steps

        batch_size, subband_dim = x.shape

        # Initialize normalization and linear layers dynamically
        if not hasattr(self, 'normalization') or self.normalization is None:
            self.normalization = nn.LayerNorm(subband_dim).to(x.device)
        if self.linear is None:
            self.linear = nn.Linear(subband_dim, self.hidden_dim).to(x.device)

        # Normalize input
        x = self.normalization(x)

        # Initialize membrane potentials for the spiking neuron
        mem = torch.zeros((batch_size, self.hidden_dim), dtype=torch.float32, device=x.device)

        # Record the outputs
        spk_rec = []
        mem_rec = []

        for step in range(num_steps):
            cur = self.linear(x)  # Linear transformation
            spk, mem = self.spikingneuron(cur, mem)  # Spiking neuron dynamics
            spk_rec.append(spk)
            mem_rec.append(mem)

        # Stack the recorded values across timesteps
        spk_rec = torch.stack(spk_rec, dim=0)  # Shape: (num_steps, batch_size, hidden_dim)
        mem_rec = torch.stack(mem_rec, dim=0)  # Shape: (num_steps, batch_size, hidden_dim)

        return spk_rec, mem_rec


In [ ]:
def frequency_partition(spectrogram, num_subbands):
    """
    Splits the input tensor into subbands along the second dimension.
    Args:
    - spectrogram: Input tensor of shape (batch_size, hidden_dim).
    - num_subbands: Number of subbands to split the hidden_dim into.
    Returns:
    - subbands: List of tensors, each of shape (batch_size, subband_size).
    """
    batch_size, hidden_dim = spectrogram.shape
    subband_size = hidden_dim // num_subbands

    # Split along the hidden_dim axis
    subbands = torch.split(spectrogram, subband_size, dim=1)
    # print(f"Number of Subbands (fp func): {len(subbands)}")
    return subbands


def frequency_reconstruct(subbands):
    """
    Reconstructs the full spectrogram from processed subbands.
    Args:
    - subbands: List of tensors, each of shape (batch_size, subband_size, time_bins).
    Returns:
    - reconstructed: Tensor of shape (batch_size, frequency_bins, time_bins).
    """
    # Concatenate the processed subbands along the frequency axis
    reconstructed = torch.cat(subbands, dim=1)
    return reconstructed

In [ ]:
class DeepFilteringLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_filters=64):
        super(DeepFilteringLayer, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, num_filters, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(num_filters, num_filters, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(num_filters, output_dim, kernel_size=1)
        self.upsample = nn.Upsample(scale_factor=2, mode='linear', align_corners=True)  # Add this layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.upsample(x)  # Upsample to match original dimensions
        return x


In [ ]:
class IntegratedModel(nn.Module):
    def __init__(self, freq_bins, time_bins, hidden_dim, num_steps, num_subbands, beta=0.9):
        """
        Integrated model with FullbandModel, SubbandModels, and per-subband DeepFilteringLayers.
        Args:
        - freq_bins: Number of frequency bins in the spectrogram.
        - time_bins: Number of time bins in the spectrogram.
        - hidden_dim: Number of hidden units for the spiking neuron layer.
        - num_steps: Number of timesteps for spiking neuron simulation.
        - num_subbands: Number of frequency subbands.
        - beta: Decay parameter for the LIF neuron.
        """
        super(IntegratedModel, self).__init__()

        self.freq_bins = freq_bins
        self.time_bins = time_bins
        self.num_subbands = num_subbands

        subband_size = freq_bins // num_subbands

        # Fullband model
        self.fullband_model = FullbandModel(freq_bins, time_bins, hidden_dim, beta)

        # Subband models
        self.subband_models = nn.ModuleList([
            SubbandModel(hidden_dim, num_steps, beta)
            for _ in range(num_subbands)
        ])

        # Per-subband Deep Filtering Layers
        self.deep_filtering_layers = nn.ModuleList([
            DeepFilteringLayer(input_dim=hidden_dim, output_dim=subband_size)
            for _ in range(num_subbands)
        ])

    def forward(self, x, num_steps=10, clean_time_bins=None):
        """
        Forward pass through the IntegratedModel.
        Args:
        - x: Input tensor of shape (batch_size, frequency_bins, time_bins).
        - num_steps: Number of time steps for spiking neuron simulation.
        - clean_time_bins: Time bins of the clean tensor for interpolation (optional).
        Returns:
        - filtered_output: Tensor after deep filtering and interpolation.
        """
        # Fullband processing
        fullband_output, _ = self.fullband_model(x, num_steps)

        # Subband processing
        subbands = frequency_partition(fullband_output[-1], self.num_subbands)
        subband_outputs = [
            self.subband_models[i](subband, num_steps=num_steps)[0][-1]
            for i, subband in enumerate(subbands)
        ]

        # Per-subband deep filtering
        filtered_subbands = [
            self.deep_filtering_layers[i](subband_output.unsqueeze(-1)).squeeze(-1)
            for i, subband_output in enumerate(subband_outputs)
        ]

        # Concatenate filtered subbands along the feature dimension
        concatenated_output = torch.cat(filtered_subbands, dim=1)  # Shape: (batch_size, freq_bins)

        # Add a singleton spatial dimension for interpolation
        concatenated_output = concatenated_output.unsqueeze(-1)  # Shape: (batch_size, freq_bins, 1)

        # Use clean_time_bins for interpolation if provided
        target_time_bins = clean_time_bins if clean_time_bins is not None else self.time_bins

        # Interpolate to match the original spectrogram dimensions
        interpolated_output = torch.nn.functional.interpolate(
            concatenated_output, size=(self.freq_bins, target_time_bins), mode='bilinear', align_corners=True
        )

        # Remove the singleton spatial dimension
        filtered_output = interpolated_output.squeeze(-1)  # Shape: (batch_size, freq_bins, target_time_bins)

        return filtered_output


In [ ]:
def si_sdr_loss(clean, enhanced, eps=1e-8):
    """
    Computes the Scale-Invariant Signal-to-Distortion Ratio (SI-SDR) Loss.
    Args:
    - clean: Ground truth clean signal (batch_size, time_steps).
    - enhanced: Enhanced (predicted) signal (batch_size, time_steps).
    - eps: Small value to avoid division by zero.
    Returns:
    - Loss value (negative SI-SDR).
    """
    # Ensure signals have zero mean
    clean = clean - torch.mean(clean, dim=-1, keepdim=True)
    enhanced = enhanced - torch.mean(enhanced, dim=-1, keepdim=True)

    # Compute scaling factor
    scale = torch.sum(clean * enhanced, dim=-1, keepdim=True) / (torch.sum(clean**2, dim=-1, keepdim=True) + eps)

    # Projection of enhanced signal onto clean signal
    projection = scale * clean

    # Compute noise (residual)
    noise = enhanced - projection

    # SI-SDR calculation with stabilization
    numerator = torch.sum(projection**2, dim=-1) + eps
    denominator = torch.sum(noise**2, dim=-1) + eps
    si_sdr = 10 * torch.log10(numerator / denominator)

    # Return negative SI-SDR as the loss
    return -torch.mean(si_sdr)

def composite_loss(filtered_output, clean_tensor, 
                   filtered_output_complex=None, clean_tensor_complex=None, 
                   alpha=0.5, beta=0.3, p=1, target_metric=4.5):
    """
    Composite loss function combining SI-SDR, proxy MetricGAN+ generator loss, and frequency loss.
    Args:
    - filtered_output: Time-domain enhanced signal.
    - clean_tensor: Time-domain clean signal.
    - filtered_output_complex: Complex spectrogram of enhanced signal.
    - clean_tensor_complex: Complex spectrogram of clean signal.
    - alpha: Weight for SI-SDR loss.
    - beta: Weight for proxy MetricGAN+ generator loss.
    - p: Power for dynamic range compression in frequency-domain loss.
    - target_metric: Ideal target metric score (e.g., 4.5 for PESQ-like proxy metric).
    Returns:
    - Combined loss value.
    """
    # SI-SDR Loss
    si_sdr = si_sdr_loss(clean_tensor, filtered_output)

    # Proxy MetricGAN+ Generator Loss
    if filtered_output_complex is not None and clean_tensor_complex is not None:
        def spectrogram_metric_loss(clean_spectrogram, enhanced_spectrogram, p=1):
            magnitude_loss = torch.mean(torch.abs(torch.abs(clean_spectrogram)**p - torch.abs(enhanced_spectrogram)**p))
            phase_loss = torch.mean(torch.abs(torch.angle(clean_spectrogram) - torch.angle(enhanced_spectrogram)))
            return magnitude_loss + phase_loss

        gen_loss = spectrogram_metric_loss(clean_tensor_complex, filtered_output_complex, p=p)
        proxy_metric_loss = (gen_loss - target_metric) ** 2
    else:
        proxy_metric_loss = 0

    # Frequency-Domain Loss
    if filtered_output_complex is not None and clean_tensor_complex is not None:
        freq_loss = torch.mean(
            torch.abs(filtered_output_complex.abs()**p - clean_tensor_complex.abs()**p)
        ) + torch.mean(
            torch.abs(filtered_output_complex - clean_tensor_complex)
        )
    else:
        freq_loss = 0

    # Composite Loss
    total_loss = alpha * si_sdr + beta * proxy_metric_loss + freq_loss
    return total_loss

In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm  # For tracking progress

# Paths to the feature and label directories
feature_dir = "E:/CS541 - Deep Learning/noisy_audio_np"
label_dir = "E:/CS541 - Deep Learning/clean_audio_np"

# Load all feature and label file paths
feature_files = sorted(os.listdir(feature_dir))
label_files = sorted(os.listdir(label_dir))

assert len(feature_files) == len(label_files), "Mismatch between feature and label file counts!"

# Model parameters
freq_bins, time_bins = 128, 860  # Based on the dimensions of your spectrograms
hidden_dim = 64                  # Hidden dimension for spiking neurons
num_steps = 10                   # Number of timesteps for spiking neurons
num_subbands = 4                 # Number of subbands (adjusted for even division)
beta = 0.9                       # Decay parameter for LIF neurons

# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IntegratedModel(freq_bins, time_bins, hidden_dim, num_steps, num_subbands, beta).to(device)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Track losses
losses = []

epochs = 3

for _ in range(epochs):
    # Training loop
    for idx in tqdm(range(len(feature_files)), desc="Processing files"):
        feature_path = os.path.join(feature_dir, feature_files[idx])
        label_path = os.path.join(label_dir, label_files[idx])

        noisy_tensor = torch.tensor(np.load(feature_path)).unsqueeze(0).to(device)
        clean_tensor = torch.tensor(np.load(label_path)).unsqueeze(0).to(device)

        # Forward pass
        filtered_output = model(noisy_tensor, num_steps=10, clean_time_bins=clean_tensor.shape[2])

        # Compute composite loss
        total_loss = composite_loss(
            filtered_output, clean_tensor,
            filtered_output_complex=None, clean_tensor_complex=None,  # Replace None if using spectrograms
            alpha=0.25, beta=0.1, p=1, target_metric=4.5
        )

        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # print(f"Total loss for file {idx + 1}: {total_loss.item()}")

# Print average loss across all files
average_loss = sum(losses) / len(losses)
print(f"\nAverage Loss across all files: {average_loss:.6f}")